# Feature extraction with tsfresh transformer

[tsfresh](https://tsfresh.readthedocs.io) is a tool for extacting summary features
from a collection of time series. It is an unsupervised transformation, and as such
can easily be used as a pipeline stage in classification, clustering and regression
in conjunction with a scikit-learn compatible estimator.

## Preliminaries
You have to install tsfresh if you haven't already. To install it, uncomment the cell below:

In [ ]:
# !pip install --upgrade tsfresh

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

from aeon.datasets import load_arrow_head, load_basic_motions
from aeon.transformations.collection.feature_based import TSFresh, TSFreshRelevant

## Example data set

We use the ArrowHead data from the [UCR TSC archive](https://timeseriesclassification.com).
as an example dataset. See
[dataset notebook](https://github.com/aeon-toolkit/aeon/blob/main/examples/datasets
/provided_data.ipynb) for more details.

In [ ]:
X_train, y_train = load_arrow_head(split="train")
X_test, y_test = load_arrow_head(split="test")
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
X_train[0]

## Using tsfresh to extract features

There are two versions of TSFresh feature extractors wrapped in aeon. The
first is the unsupervised
`TSFresh` which by default extracts all 4662 features. See the
documentation for parameter configuration.

In [ ]:
t = TSFresh()
Xt = t.fit_transform(X_train)
Xt.shape
Xt2 = t.transform(X_test)

The second is `TSFreshRelevant` which uses `y` to select the most
relevant features.

In [ ]:
t = TSFreshRelevant()
t.fit(X_train, y_train)
Xt = t.transform(X_test)
Xt.shape

## Using tsfresh with scikit estimators

You can use the tsfresh transformer with any scikit-learn compatible estimator.


In [ ]:
classifier = make_pipeline(
    TSFresh(default_fc_parameters="efficient", show_warnings=False),
    RandomForestClassifier(),
)
classifier.fit(X_train, y_train)
classifier.score(X_test, y_test)

For convenience and consistency of use we also have hard coded TSFresh classifier,
regressor and clusterer.

In [ ]:
from aeon.classification.feature_based import TSFreshClassifier
from aeon.clustering.feature_based import TSFreshClusterer

cls = TSFreshClassifier()
clst = TSFreshClusterer()

cls.fit(X_train, y_train)
cls.score(X_test, y_test)
clst.fit(X_train)
print(cls.predict(X_test))
print(clst.predict(X_test))

[7 1 4 4 0 4 4 5 2 1 5 7 4 4 5 0 0 6 6 0 6 4 5 6 3 6 7 1 6 4 4 1 5 0 4 4 7
 6 6 2 1 0 0 4 6 5 4 6 4 6 6 0 4 6 1 1 4 1 4 1 4 0 1 4 1 5 4 7 4 7 6 4 6 1
 6 4 6 7 4 6 6 1 6 1 4 7 6 4 6 0 4 6 4 6 6 4 0 3 4 6 4 1 0 0 4 4 6 1 0 7 4
 6 0 4 4 0 1 6 6 0 2 0 6 0 3 6 5 7 6 4 4 3 6 6 6 1 7 4 6 6 4 4 6 6 0 4 6 4
 5 0 4 4 6 4 6 1 5 6 6 0 6 0 3 4 4 6 1 5 3 7 6 6 6 7 4]


By default, the `TSFreshClassifier` uses the supervised
`TSFreshRelevant` and the scitkit `RandomForestClassifier`.
 You can
change this through the constructor

In [ ]:
from aeon.classification.sklearn import RotationForestClassifier

cls = TSFreshClassifier(
    relevant_feature_extractor=False, estimator=RotationForestClassifier(n_estimators=5)
)  #
cls.fit(X_train, y_train)
cls.score(X_test, y_test)

0.5771428571428572

By default, the `TSFreshClusterer` uses the unsupervised `TSFresh`
and the `sklearn` clusterer `KMeans` with default parameters (which fits 8 clusters).
 You can also configure this through the constructor.

In [ ]:
from sklearn.cluster import KMeans

clst = TSFreshClusterer(estimator=KMeans(n_clusters=3))
clst.fit(X_train)
print(clst.predict(X_test))

[1 0 1 1 1 1 1 0 2 0 0 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1
 1 1 2 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 1 2 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 2 1 1 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 0 0 0 1 1 1 1 1 1]


The `TSFreshRegressor` uses the supervised
`TSFreshRelevant` and the scitkit `RandomForestRegressor`.

In [ ]:
from aeon.regression.feature_based import TSFreshRegressor

reg = TSFreshRegressor()
from aeon.datasets import load_covid_3month

X, y = load_covid_3month()
reg.fit(X, y)

TSFreshRegressor()

TSFreshRegressor()

## TSFresh with multivariate time series data

All three estimators can be used with multivariate time series. The estimators
calculate the features on each channel independently then concatenate the results.
The full transform creates `777*n_channels` features.

In [ ]:
X_train, y_train = load_basic_motions(split="train")
X_test, y_test = load_basic_motions(split="test")
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(40, 6, 100) (40,) (40, 6, 100) (40,)


In [ ]:
tsfresh = TSFresh()
X = tsfresh.fit_transform(X_train, y_train)
X.shape

In [ ]:
cls = TSFreshClassifier()
clst = TSFreshClusterer(estimator=KMeans(n_clusters=4))
cls.fit(X_train, y_train)
cls.score(X_test, y_test)
clst.fit(X_train)
print(cls.predict(X_test))